In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import sys
import pandas as pd
import scipy
from scipy import stats
import netCDF4
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib import rcParams
import cartopy.crs as ccrs
from sklearn.preprocessing import StandardScaler
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
import joblib
import glob

In [4]:
### create lists of model groups
### all possible CMIP5/CMIP6 models
model_list = ['ACCESS1-0', 'ACCESS1-3', 'ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'bcc-csm1-1_',
             'bcc-csm1-1-m','BCC-CSM2-MR_', 'BCC-ESM1', 'BNU-ESM', 'CAMS-CSM1-0', 'CanESM2', 'CanESM5',
             'CCSM4', 'CESM2_', 'CESM2-WACCM', 'CNRM-CM5', 'CNRM-CM6-1','CNRM-ESM2-1','CSIRO-Mk3-6-0', 'E3SM-1-0', 'EC-Earth3-Veg', 'EC-EARTH', 'FGOALS-f3-L', 'FGOALS-g2', 'FGOALS-g3', 
             'GFDL-CM3', 'GFDL-CM4', 'GFDL-ESM2G', 'GFDL-ESM2M', 'GFDL-ESM4',
             'GISS-E2-1-G', 'GISS-E2-1-H', 'GISS-E2-H', 'GISS-E2-R', 'HadGEM2-ES', 'HadGEM3-GC31-LL',
             'HadGEM3-GC31-MM', 'inmcm4', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM5A-MR', 'IPSL-CM5B-LR', 'IPSL-CM6A-LR',
             'MIROC5', 'MIROC6', 'MIROC-ES2L', 'MIROC-ESM', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MPI-ESM-P', 'MRI-CGCM3',
             'MRI-ESM2-0', 'NESM3', 'NorESM1-M', 'NorESM2-LM', 'NorESM2-MM', 'SAM0-UNICON', 'UKESM1-0-LL']
### just CMIP5
model_list_cmip5 = ['ACCESS1-0', 'ACCESS1-3', 'bcc-csm1-1_', 'bcc-csm1-1-m', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'EC-EARTH', 'FGOALS-g2', 'GFDL-CM3', 
               'GFDL-ESM2G', 'GFDL-ESM2M', 'GISS-E2-H', 'GISS-E2-R', 'HadGEM2-ES', 'inmcm4', 'IPSL-CM5A-MR', 'IPSL-CM5B-LR', 'MIROC5', 'MIROC-ESM', 'MPI-ESM-LR', 'MPI-ESM-MR', 
                    'MPI-ESM-P', 'MRI-CGCM3', 'NorESM1-M']
### list for nicer labels later
model_list_cmip5_labels = ['ACCESS1-0', 'ACCESS1-3', 'BCC-CSM1-1', 'BCC-CSM1-1-m', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'EC-EARTH', 'FGOALS-g2', 
                           'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'GISS-E2-H', 'GISS-E2-R', 'HadGEM2-ES', 'INM-CM4', 'IPSL-CM5A-MR', 'IPSL-CM5B-LR', 'MIROC5', 'MIROC-ESM', 'MPI-ESM-LR', 'MPI-ESM-MR',
                           'MPI-ESM-P', 'MRI-CGCM3', 'NorESM1-M']
### just CMIP6
model_list_cmip6 = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'BCC-CSM2-MR_', 'BCC-ESM1', 'CAMS-CSM1-0', 'CanESM5',
             'CESM2_', 'CESM2-WACCM', 'CNRM-CM6-1','CNRM-ESM2-1', 'E3SM-1-0', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'FGOALS-g3', 
             'GFDL-CM4', 'GFDL-ESM4', 'GISS-E2-1-G', 'GISS-E2-1-H', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR',
             'MIROC6', 'MIROC-ES2L', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3', 'NorESM2-LM', 'NorESM2-MM', 'SAM0-UNICON', 'UKESM1-0-LL']
model_list_cmip6_labels = ['ACCESS-CM2', 'ACCESS-ESM1-5', 'AWI-CM-1-1-MR', 'BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0', 'CanESM5',
             'CESM2', 'CESM2-WACCM', 'CNRM-CM6-1','CNRM-ESM2-1', 'E3SM-1-0', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'FGOALS-g3', 
             'GFDL-CM4', 'GFDL-ESM4', 'GISS-E2-1-G', 'GISS-E2-1-H', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR',
             'MIROC6', 'MIROC-ES2L', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0', 'NESM3', 'NorESM2-LM', 'NorESM2-MM', 'SAM0-UNICON', 'UKESM1-0-LL']
### a couple more lists needed later for models that don't match SWOOSH data availability exactly in time
model_list_cmip5_1968 = ['MPI-ESM-P']
model_list_cmip6_1977 = ['CESM2_', 'E3SM-1-0', 'EC-Earth3-Veg', 'FGOALS-f3-L', 'FGOALS-g3', 'GISS-E2-1-H', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'INM-CM4-8', 
                         'INM-CM5-0', 'MIROC-ES2L', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'NESM3', 'NorESM2-LM', 'NorESM2-MM', 'SAM0-UNICON']

In [5]:
print(len(model_list))
print(len(model_list_cmip5))
print(len(model_list_cmip6))

61
27
34


In [6]:
### all models have been interpolated to the same 5x5 degree grid for temperature (=predictors in ridge regressions)
lat = netCDF4.Dataset('./data/cmip/ta/ta_Amon_ACCESS-CM2_historical-ssp_r1i1p1f1_gn.nc')['lat'][:]

In [7]:
### define indices to select temperature predictors within 60N-60S
print(lat[6:30])
la=6
lb=30

[-57.5 -52.5 -47.5 -42.5 -37.5 -32.5 -27.5 -22.5 -17.5 -12.5  -7.5  -2.5
   2.5   7.5  12.5  17.5  22.5  27.5  32.5  37.5  42.5  47.5  52.5  57.5]


In [8]:
### load 50 realisations of tropical 68 hPa SWOOSH (reflecting sampling and observational uncertainty)
swoosh_tropics = netCDF4.Dataset('./data/swoosh/SWOOSH_samples_scaleSE_1.0_scaleB_1.0_nsamples_50_aura5_finalNatGeo.nc')['samples'][:,:]
non_nan_indices = np.argwhere(~np.isnan(swoosh_tropics[:,0])).T
### only 315 monthly samples will actually be selected (only months where actual observations in the tropics, 68 hPa are available)
print(non_nan_indices[:])
print(non_nan_indices[:].shape)

[[ 72  86  87  88  92  93  94  95  96  97  98  99 100 101 102 103 104 105
  106 107 108 109 110 111 114 116 117 118 120 121 122 123 124 126 127 128
  129 130 132 134 135 136 138 140 141 142 144 146 147 150 152 153 154 155
  156 157 158 159 161 163 164 165 167 168 169 170 171 173 174 175 176 177
  179 181 182 183 185 187 188 189 191 193 194 195 197 199 200 201 203 205
  206 207 209 211 212 213 214 215 217 218 219 220 221 223 226 228 229 230
  232 233 234 235 237 238 240 241 244 246 247 248 249 250 251 252 253 254
  255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272
  273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290
  291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308
  309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326
  327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344
  345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362
  363 364 365 366 367 368 369 370 371 

In [9]:
### nr_lat after selection
nr_lat=len(lat[la:lb])
nr_lon=72
nr_models=len(model_list)
### up to seven vertical levels were used in the test = #"planes" (i.e. is the vertical dimension of the temperature data read in later)
nr_planes=7

In [11]:
### define range for the regluarization parameter to be searched over
### it doesn't harm for this to be a large range; main point is that the alpha that is eventually selected falls within the range
### and one might also want to set a certain level of minimum regularization
alpha_i=[0.0001,0.0003,0.001,0.003,0.01,0.03,0.1,0.3,1,2,4,6,8,10,12,14,16,18,20,25,30,40,50,60,70,80,90,100,150,200,250,300,400,500,600,700,800,900,1000,3000,5000,10000,12000,14000,16000,18000,20000,25000,30000,50000,100000,300000,1e6,3e6,1e7,3e7,1e8]

In [12]:
### prepare for data storage
dict_regr_results = {}

In [13]:
### empty arrays to later store the learned coefficients/parameters, the ridge predictions, and predicted values under 4xCO2
### note we use only five vertical levels here for temperature
coeffs = np.empty((nr_models,nr_planes-2,nr_lat,nr_lon))
y_true_all = np.empty((nr_models))
y_pred_all = np.empty((nr_models))
y_true_all_4xco2 = np.empty((nr_models))
y_pred_all_4xco2 = np.empty((nr_models))

In [14]:
nt_train=len(swoosh_tropics[:,0])
print(nt_train)
### define the lag for the temperature to be considered over (in months)
lag=2

444


In [15]:
### empty arrays to later save the change in ppmv per degree warming under 4xCO2, calculated from annually averaged data
feedbacks_true_am = np.empty((nr_models))
feedbacks_pred_am = np.empty((nr_models))

In [16]:
#### shift non-nan by lag; lose the first nr lag time steps (need past temperature information to predict current SWV values)
non_nan_indices_lag = non_nan_indices-2
print(non_nan_indices_lag)

[[ 70  84  85  86  90  91  92  93  94  95  96  97  98  99 100 101 102 103
  104 105 106 107 108 109 112 114 115 116 118 119 120 121 122 124 125 126
  127 128 130 132 133 134 136 138 139 140 142 144 145 148 150 151 152 153
  154 155 156 157 159 161 162 163 165 166 167 168 169 171 172 173 174 175
  177 179 180 181 183 185 186 187 189 191 192 193 195 197 198 199 201 203
  204 205 207 209 210 211 212 213 215 216 217 218 219 221 224 226 227 228
  230 231 232 233 235 236 238 239 242 244 245 246 247 248 249 250 251 252
  253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270
  271 272 273 274 275 276 277 278 279 280 281 282 283 284 285 286 287 288
  289 290 291 292 293 294 295 296 297 298 299 300 301 302 303 304 305 306
  307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324
  325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342
  343 344 345 346 347 348 349 350 351 352 353 354 355 356 357 358 359 360
  361 362 363 364 365 366 367 368 369 

In [17]:
## target figure x/y ratio
plt.rcParams['figure.figsize'] = [20, 4]
### now fit individual ridge regressions for the CMIP models
### make list to keep track of best alpha values. Needed to adjust search range if current one is not sufficient
alpha_list=[]
### select months that are not masked in SWOOSH, but for temperature (includes lags)
months = np.array(non_nan_indices_lag.flatten())
### run this once for all 61 CMIP5/CMIP6 models
for modeli in range(0,nr_models):
    print(model_list[modeli])
    ### load temperature ta data for model i on five pressure levels
    ### note this is preprocessed data using cdo operators. The data folder will be made available ('Data Availability' statement)
    ### using ncdump -h you can trace the series of commands (selection of vertical levels, time period, and regridding for all files, depending on the CMIP model and its data availability)
    ### For one of the MIROC models vertical levels needed to be manually sub-selected due to a higher vertical resolution provided in the CMIP output
    X_raw_hist = netCDF4.Dataset(glob.glob('./data/cmip/ta/ta*'+model_list[modeli]+'*historical*.nc')[0])['ta'][:nt_train,1:nr_planes-1,la:lb,:]
    pressure_levels = netCDF4.Dataset(glob.glob('./data/cmip/ta/ta*'+model_list[modeli]+'*historical*.nc')[0])['plev'][1:nr_planes-1]
    # check we really fetched the correct levels etc for all models
#     print(X_raw_hist.shape)
#     print(pressure_levels)
    X_raw_hist = X_raw_hist.reshape(nt_train,nr_lon*nr_lat*(nr_planes-2))
    # redundant renaming; useful line if other transformations of temperature shall be used as well for prediction
    ### add lags...two months into the past turned out empirically to be the best compromise
    ### note the lag introduced here, which is accounted for above for the later masking of indices already
    X_lag_hist = np.hstack((X_raw_hist[lag:,:],X_raw_hist[lag-1:-1,:],X_raw_hist[lag-2:-2,:]))
#     print(X_lag_hist.shape)
    ### index to select 70 hPa SWV data from the CMIP simulations (files also contain 100 hPa)
    level_selected=1
    ### now load SWV data; convert kg/kg air to volume mixing ratios in ppmv
    ### previous cdo commands: time merging and selection (model-dependent), selection of two pressure levels (100, 70 hPa), zonal and 30N-30S averaging
    Y_raw_hist = netCDF4.Dataset(glob.glob('./data/cmip/hus/hus*'+model_list[modeli]+'*historical*.nc')[0])['hus'][:nt_train,level_selected,0,0]*1e6/0.6213
    ### subselect samples according to SWOOSH mask and when simulations start (not all timelines are the same due to missing data, see Online Methods)
    if model_list[modeli] in model_list_cmip5_1968:
### first 192 months can be considered fully covered for the model, similar to the last 192 months in SWOOSH. Critical is the masking from 72+192 onwards; covering Pinatubo
        shifted_months = months[:-192]+192
        months_modeli = np.concatenate([np.arange(0,192),shifted_months])
    elif model_list[modeli] in model_list_cmip6_1977:
        shifted_months = months[:-84]+84
        months_modeli = np.concatenate([np.arange(0,84),shifted_months])
    else:
        months_modeli = months
    ### take logarithm of SWV vmr, since we know of the approx. exponential dependency on temperature; other approximative transformations (and combinations thereof) 
    ### have been attempted but did not perform better
    Y_hist = np.log(Y_raw_hist)
    ### create dictionary to save all final functions for easy recovery later
    regressors_col = {}
    ### define parameters for ridge regression, including alpha range to be searched over (see above)
    parameters = {
    'alpha': alpha_i,
    'fit_intercept': [True],
    'max_iter':[1000],
    'random_state':[100]
             }
    ### define object for k-fold cross-validation; keep time order as there is lagged structure in the data
    cv_obj = KFold(n_splits=5,shuffle=False)
    ### drop unobserved samples before fitting the scaler
    X_lag_hist = X_lag_hist[months_modeli,:]
    ### apply standard scaling to the temperature data
    scaler = StandardScaler()
    X_train_norm = scaler.fit_transform(X_lag_hist[:,:])
    del X_raw_hist, X_lag_hist, Y_raw_hist
    ### define regression/cross-validation object
    regr = GridSearchCV(Ridge(),parameters,cv=cv_obj,n_jobs=-1,refit=True)
    ### select hus - minus lag (there is no temperature data to predict) / does not reduce sample size as the first 1984 hus samples are not in SWOOSH anyway
    Y_hist = Y_hist[lag:]
    ### now select observed indices; already subtracted lag in cell above to accommodate for the shift
    Y_hist = Y_hist[months_modeli]
    ### now fit ridge regression, including cross-validation
    regr.fit(X_train_norm[:,:],Y_hist[:])
    ### save regr model 
    dict_regr_results[model_list[modeli]] = [regr]
    print(regr.best_estimator_)
    alpha_list.append(regr.best_estimator_.alpha)
    Y_pred_train = regr.best_estimator_.predict(X_train_norm)

    ### now load and pre-process the abrupt-4xCO2 data for the same model accordingly
    nt_4xco2 = 1800
    X_raw_4xco2 = netCDF4.Dataset(glob.glob('./data/cmip/ta/ta*'+model_list[modeli]+'*4xCO2*.nc')[0])['ta'][:nt_4xco2,1:nr_planes-1,la:lb,:]
    ### appearingly strange line follows, but one model does not have the full 150 years
    nt_4xco2 = X_raw_4xco2.shape[0]
    X_raw_4xco2 = X_raw_4xco2.reshape(nt_4xco2,nr_lon*nr_lat*(nr_planes-2))
    X_lag_4xco2 = np.hstack((X_raw_4xco2[lag:,:],X_raw_4xco2[lag-1:-1,:],X_raw_4xco2[lag-2:-2,:]))
    Y_raw_4xco2 = netCDF4.Dataset(glob.glob('./data/cmip/hus/hus*'+model_list[modeli]+'*4xCO2*.nc')[0])['hus'][:nt_4xco2,level_selected,0,0]*1e6/0.6213
    ### apply scaling of predictors that is consistent with the one for the historical training data
    X_lag_4xco2_norm = scaler.transform(X_lag_4xco2)
    ### predict monthly mean 4xCO2 Delta/response
    Y_pred_4xco2 = np.exp(regr.predict(X_lag_4xco2_norm))
    ### load globally averaged surface temperature (to regress against the predicted/true changes in hus)
    tas_4xco2 = netCDF4.Dataset(glob.glob('./data/cmip/tas/tas_global_mean/tas*'+model_list[modeli]+'*4xCO2*.nc')[0])['tas'][:nt_4xco2,0,0]
    ### prepare annual averaging of data. Since first two months drop our due to lag, start after the first year
    Y_pred_4xco2_am = np.empty((int((Y_pred_4xco2.shape[0]-12+lag)/12)))
    Y_raw_4xco2_am = np.empty((int((Y_raw_4xco2.shape[0]-12)/12)))
    tas_4xco2_am = np.empty((int((tas_4xco2.shape[0]-12)/12)))
    for i in np.arange(0,int(Y_pred_4xco2.shape[0]-12+lag),12):
        Y_pred_4xco2_am[int(i/12)] = np.mean(Y_pred_4xco2[12-lag+i:24-lag+i],axis=0)
        Y_raw_4xco2_am[int(i/12)] = np.mean(Y_raw_4xco2[12+i:24+i],axis=0)
        tas_4xco2_am[int(i/12)] = np.mean(tas_4xco2[12+i:24+i],axis=0)
    ### estimate true feedback (slope) in ppmv/degree Kelvin warming
    res_mm_true_am = stats.linregress(tas_4xco2_am[:],Y_raw_4xco2_am[:])
    ### now the same for the predicted feedback
    res_mm_pred_am = stats.linregress(tas_4xco2_am[:],Y_pred_4xco2_am[:])
    ### save the results
    feedbacks_true_am[modeli] = res_mm_true_am.slope
    feedbacks_pred_am[modeli] = res_mm_pred_am.slope

ACCESS1-0
Ridge(alpha=18000, max_iter=1000, random_state=100)
ACCESS1-3
Ridge(alpha=5000, max_iter=1000, random_state=100)
ACCESS-CM2
Ridge(alpha=1000, max_iter=1000, random_state=100)
ACCESS-ESM1-5
Ridge(alpha=3000, max_iter=1000, random_state=100)
AWI-CM-1-1-MR
Ridge(alpha=1000, max_iter=1000, random_state=100)
bcc-csm1-1_
Ridge(alpha=3000, max_iter=1000, random_state=100)
bcc-csm1-1-m
Ridge(alpha=100, max_iter=1000, random_state=100)
BCC-CSM2-MR_
Ridge(alpha=500, max_iter=1000, random_state=100)
BCC-ESM1
Ridge(alpha=1000, max_iter=1000, random_state=100)
BNU-ESM
Ridge(alpha=0.003, max_iter=1000, random_state=100)
CAMS-CSM1-0
Ridge(alpha=3000, max_iter=1000, random_state=100)
CanESM2
Ridge(alpha=12000, max_iter=1000, random_state=100)
CanESM5
Ridge(alpha=3000, max_iter=1000, random_state=100)
CCSM4
Ridge(alpha=3000, max_iter=1000, random_state=100)
CESM2_
Ridge(alpha=5000, max_iter=1000, random_state=100)
CESM2-WACCM
Ridge(alpha=1000, max_iter=1000, random_state=100)
CNRM-CM5
Ridge(a

In [18]:
dict_regr_results['y_true_all'] = y_true_all
dict_regr_results['y_pred_all'] = y_pred_all
dict_regr_results['feedbacks_true_am'] = feedbacks_true_am
dict_regr_results['feedbacks_pred_am'] = feedbacks_pred_am
# dict_regr_results['forcings_true_am'] = forcings_true_am
# dict_regr_results['forcings_pred_am'] = forcings_pred_am

In [19]:
joblib.dump(dict_regr_results,'./data/results//models_lag2_5planes_60NS_all_models_final.pkl')

['./data/results//models_lag2_5planes_60NS_all_models_final.pkl']